In [1]:
import pandas as pd
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
bio_df = pd.read_csv('./data/bio_sample.csv')

In [3]:
bio_df.head(2)

,Title,profileName,review/score,review/summary,review/text,description,authors,image,categories,ratingsCount
0,"Genghis Khan: Life, Death and Resurrection",P. Carey,2.0,Not a great book,Read both this book and Genghis Khan and the M...,"Genghis Khan is one of history's immortals, al...",John Man,http://books.google.com/books/content?id=0eEKA...,Biography & Autobiography,3.0
1,Queen Victoria,"Linda Baker ""quiltsbylin""",5.0,Masterful & a charming piece of true history,I went into this book after watching most of t...,Commemorating the centennial of Queen Victoria...,Christopher Hibbert,http://books.google.com/books/content?id=uWlnA...,Biography & Autobiography,5.0


In [4]:
X = bio_df['description']
y = bio_df['Title']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
def my_lemmatizer(text):
    wnet = WordNetLemmatizer()
    return [wnet.lemmatize(w) for w in text.split()]

In [7]:
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [8]:
wnet = WordNetLemmatizer()
lem_stopwords = [wnet.lemmatize(w) for w in stopwords.words('english')]
new_stopword = 've'

lem_stopwords = lem_stopwords + [wnet.lemmatize(new_stopword)]

In [9]:
pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 1_000)),
    ('mnb', MultinomialNB(alpha = 0.5))
])

In [36]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('tf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'it', 'itself', ...],
                                 token_pattern=None,
                                 tokenizer=<function my_lemmatizer at 0x7feb0a1bdb80>)),
                ('mnb', MultinomialNB(alpha=0.5))])

In [37]:
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.6645066666666667, 0.61568)

In [38]:
with open('fiction_pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)

### opening a pickled model

In [10]:
fp = "my-capstone/book_recommendation/streamlit_app/pickled_models/bio_pipe.pkl"

In [12]:
with open('bio_pipe.pkl', 'rb') as f:
    model = pickle.load(f)

In [21]:
import numpy as np

In [20]:
user_text = input()

 anne 


In [43]:
user_text = input('Enter some text please: ')
prediction = model.predict([user_text])
print(f'You should read: {prediction}')

Enter some text please:  self love


You should read: ['Left to Tell: Discovering God Amidst The Rwandan Holocaust']


In [45]:
bio_df['Title'].value_counts()

It's Not About the Bike: My Journey Back to Life                             1162
The Princess Bride                                                            980
John Adams                                                                    785
Confessions of an Economic Hitman                                             748
All Creatures Great and Small                                                 553
                                                                             ... 
THE BOY IN THE GREEN SUIT                                                       1
The God I Love: A Lifetime of Walking with Jesus                                1
MARY QUEEN OF SCOTS                                                             1
Ronald Reagan: Our Fortieth President (Spirit of America: Our Presidents)       1
Bruce Lee: The Incomparable Fighter                                             1
Name: Title, Length: 2309, dtype: int64